In [1]:
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights, homonyms, skill_cat, pandas as pd

In [2]:
skill_cat._refresh()
homonyms._refresh()

In [3]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)
# job_descriptions = skill_weights.get_job_descriptions(raw_job_details)

In [4]:
len(jobs)

100

In [5]:
jobs.head(3)

,company_name,location,role,url,excitement
id,,,,,
bd59469c-29f4-41b9-9001-5b7902d766e1,Ansys,"Madrid, Community of Madrid, Spain",R&D Engineer II - Python f/m,https://www.linkedin.com/jobs/search/?currentJ...,4
0124e218-2d33-41e8-907b-5228ea386455,Treibacher Industrie AG,"Althofen, Carinthia, Austria",AI/ML Specialist – Software Engineer (m/f/d),https://www.linkedin.com/jobs/search/?geoId=91...,4
35b60a7d-f9a3-4f63-b8ab-4db8db49fca8,Helsing,"Paris, Île-de-France, France",Deployed AI Engineer,https://www.linkedin.com/jobs/search/?currentJ...,5


# What is the goal here?
How well do we categorize the skills in listings?

In [6]:
job_skills_data = skill_weights.get_job_skills_data(raw_job_details=raw_job_details, collapse_teal_names=False)
job_skills_data["category"] = [skill_cat.skill_to_categories.get(skill, {"UNK"}) for skill in job_skills_data.index.get_level_values(2)]
jsd_count = job_skills_data.pop("count")
job_skills_data["share of job"] = jsd_count / (job_skills_data["category"].str.len() * jsd_count.groupby(level=0).sum())
job_skills_data = job_skills_data.explode("category").reset_index().set_index(["id", "teal category", "category", "skill", "tealPhrase"]).sort_index()

In [7]:
job_skills_data

share of job
id                                   teal category        category skill                 tealPhrase                        
0124e218-2d33-41e8-907b-5228ea386455 emphasis             UNK      excellent             excellent                 0.018519
                                                                   knowledge             knowledge                 0.037037
                                                                   managing requirements requirements              0.018519
                                     functional abilities Academic computer science      computer science          0.009259
                                                          Admin    automation            automating                0.018519
...                                                                                                                     ...
fdd15faa-0804-4b07-9f55-715436213e75 general abilities    UNK      resilience            resilience                0.007874
                                                                   strong relationships  strong relationships      0.007874
                                                                   team performance      team performance          0.007874
                                                                   trusted               trusted                   0.007874
                                     platform             Code     python                python                    0.007874

[6248 rows x 1 columns]

In [8]:
global_shares = (job_skills_data.groupby(level=[1,2,3,4]).sum() / job_skills_data.sum())["share of job"].sort_values(ascending=False)

In [9]:
global_shares

teal category         category  skill             tealPhrase             
emphasis              UNK       experience        experience                 0.063014
general abilities     Data      data              data                       0.054511
functional abilities  ML        machine learning  machine learning           0.036887
                                                  ai                         0.019050
general abilities     Academic  research          research                   0.017116
                                                                               ...   
functional abilities  UNK       dissemination     dissemination              0.000054
general abilities     UNK       special projects  special projects           0.000054
functional abilities  UNK       process analysis  process analysis           0.000054
                      Unix      infrastructure    scalable infrastructure    0.000040
                      Admin     infrastructure    scalable infrast

In [16]:
global_shares.loc[:, :, :, "bash"].sum()

KeyError: 'bash'

In [10]:
global_shares.loc[:, "UNK"].groupby("skill").sum().sort_values(ascending=False).head(50)

skill
experience                  0.063014
deep                        0.012099
knowledge                   0.011948
strong                      0.010628
managing requirements       0.009255
excellent                   0.005845
white space                 0.005680
ability                     0.005307
global                      0.005139
create                      0.005069
key                         0.004945
cutting edge                0.004256
partner                     0.004079
challenges                  0.004038
statistics                  0.004037
innovative                  0.003997
passionate                  0.003870
responsible                 0.003720
paid                        0.003701
highly                      0.003692
libraries                   0.003394
must                        0.003356
computer vision             0.003338
at least                    0.003158
own                         0.003145
experts                     0.003145
preferred                   0.00